In [1]:
# Original Source: https://github.com/pytorch/examples/blob/master/mnist/main.py
# Code from: https://github.com/jiapei100/PyTorchZeroToAll/blob/master/10_1_cnn_mnist.py

from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

First of all, import all needed packages, particularly: **torch** and **torchvision**.

Loading MNIST dataset. You'll see 4 datasets are downloaded during the running process.
* Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
* Downloading http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
* Downloading http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
* Downloading http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz


In [2]:

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


Afterwards, define CNN


In [3]:

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x, dim=1)


model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


Define training function

In [4]:

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


Define testing function

In [5]:

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, requires_grad=True), Variable(target)
        with torch.no_grad():
            output = model(data)
            # sum up batch loss
            test_loss += F.nll_loss(output, target, size_average=False).item()
            # get the index of the max log-probability
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Train and test

In [ ]:

for epoch in range(1, 10):
    train(epoch)
    test()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303555
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.290246
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.286248
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.261360
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.274368
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.260666
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.236734
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.215869
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.155725
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.128953
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.022502
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.949315
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.895889
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.718923
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.595432
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.299183
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.115363
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.910529
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.752183
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.839759
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.180942
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.047357
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.108087
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.041769
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.181440
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.162244
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.055101
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.058011
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.108329
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.107003
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.046885
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.142366
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.273130
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.165347
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.085068
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.059307
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.097881
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.180811
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.062799
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.080875


Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.142366
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.028229
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.091747
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.098418
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.092626
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.148515
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.181647
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.090213
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.284534
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.027387
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.097889
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.124988
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.044127
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.110733
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.071159
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.164171
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.068082
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.034909
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.076453
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.048231


Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.096610
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.037901
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.101248
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.041605
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.026093
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.025273
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.091183
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.038097
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.141373
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.076695
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.090809
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.069621
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.148963
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.133184
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.054068
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.090830
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.007456
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.011075
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.036813
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.084678


Finally, we copy and paste the last 5 lines of the testing result:

* Train Epoch: 9 [56960/60000 (95%)]	Loss: 0.005480
* Train Epoch: 9 [57600/60000 (96%)]	Loss: 0.103070
* Train Epoch: 9 [58240/60000 (97%)]	Loss: 0.003925
* Train Epoch: 9 [58880/60000 (98%)]	Loss: 0.018122
* Train Epoch: 9 [59520/60000 (99%)]	Loss: 0.025634

*Test set: Average loss: 0.0484, Accuracy: 9849/10000 (98%)

**NOTE**: The above result changes every time, but fianl Accuracy is always roughly the same.